# IACS Capstone Project Team Spotify 1: Playlist Prediction
### Spotify API dataset creation

In [19]:
import numpy as np
import pandas as pd
import spotipy
import spotipy.util as util
from spotipy.oauth2 import SpotifyClientCredentials
import sys
from sklearn.metrics import mean_squared_error
from math import sqrt

### Authentication

In [20]:
client_credentials_manager = SpotifyClientCredentials(client_id='df846cfd28e745178054587b3484f91c',                                                client_secret='e3d39fc92a954e028ff1490288f3fe5c')
sp = spotipy.Spotify(client_credentials_manager=client_credentials_manager)

In [21]:
#sp.audio_features(tracks=item['track']['id'])
featured_names = []
featured_playlists = sp.featured_playlists(locale=None, country=None, timestamp=None, limit=50, offset=0)
for i, playlist in enumerate(featured_playlists['playlists']['items']):
    featured_names.append(playlist['name'])
featured_names

[u'New Music Friday',
 u'TGIF',
 u'Massive Pop Remixes',
 u'Young, Wild & Free',
 u'Old School Hip Hop House Party',
 u'Piano in the Background',
 u'Monster Dance Hits',
 u'Ambient Chill',
 u'Alternative R&B',
 u'Late Night Indie',
 u'Teen Party',
 u'Throwback Jams',
 u'Evening Chill',
 u'Mellow Beats',
 u'Are & Be',
 u'Mellow Nights',
 u'Night Rider',
 u'Chilled R&B']

In [22]:
import time

class Timer(object):
    def __init__(self, verbose=True):
        self.verbose = verbose

    def __enter__(self):
        self.start = time.time()
        return self

    def __exit__(self, *args):
        self.end = time.time()
        self.secs = self.end - self.start
        self.msecs = self.secs * 1000  # millisecs
        if self.verbose:
            print('elapsed time: %f ms %f projected finish' % (self.msecs, self.msecs*1726/(4*1000*3600)))


In [41]:
def chunks(seq, num):
    avg = len(seq) / float(num)
    out = []
    last = 0.0
    
    while last < len(seq):
        out.append(seq[int(last):int(last + avg)])
        last += avg
        
    return out

final_playlists = []

In [99]:
def getPlaylistData(offset, numLoops, limit = 1):
    print('Starting...')
    
    followers = []
    names = []
    playlist_ids = []
    total_tracks = []
    featured = []
    
    t_feature_median = []

    p1_feature_median = []
    p2_feature_median = []
    p3_feature_median = []
    p4_feature_median = []
    
    popularity = []
    
    p1_popularity = []
    p2_popularity = []
    p3_popularity = []
    p4_popularity = []
    
    mean_num_markets = []
    
    p1_mean_num_markets = []
    p2_mean_num_markets = []
    p3_mean_num_markets = []
    p4_mean_num_markets = []
    
    #t_release_date = []
    
    #p1_release_date = []
    #p2_release_date = []
    #p3_release_date = []
    #p4_release_date = []
    
    for loopIndex in range(numLoops):
        
        playlists = sp.user_playlists('spotify', limit = limit, \
                                     offset = offset + limit*loopIndex)
        for i, playlist in enumerate(playlists['items']):
            try:
                with Timer() as t:
                    
                    metadata = sp.user_playlist('spotify', playlist_id = playlist['id'],\
                                               fields = 'followers.total,tracks.items(added_at,track.available_markets,\
                                               track.popularity,track.name,track.id)')
                    
                    parts = chunks(metadata['tracks']['items'],4)
                    
                    p1 = parts[0]
                    p2 = parts[1]
                    p3 = parts[2]
                    p4 = parts[3]
                    
                    t_ids = []
                    t_popularities = []
                    t_num_markets = []
                    
                    for ix, item in enumerate(metadata['tracks']['items']):
                        t_ids.append(item['track']['id'])
                        t_popularities.append(item['track']['popularity'])
                        t_num_markets.append(len(item['track']['available_markets']))
                        
                    #t_release_dates = []
                    #for ids in t_ids:
                    #    track = sp.track(str(ids))
                    #    album = sp.album(track['album']['uri'])
                    #    t_release_dates.append(int(album['release_date'][0:4]))
                    
                    #t_release_date.append(np.mean(t_release_dates))
                
                    t_features = sp.audio_features(tracks = t_ids)
                    
                    feat_len = len(metadata['tracks']['items'])
                    t_feature_matrix = np.empty((feat_len,12))
                    for ix, song in enumerate(t_features):
  
                        t_feature_matrix[ix][0] = song['acousticness']
                        t_feature_matrix[ix][1] = song['danceability']
                        t_feature_matrix[ix][2] = song['energy']
                        t_feature_matrix[ix][3] = song['instrumentalness']
                        t_feature_matrix[ix][4] = song['key']
                        t_feature_matrix[ix][5] = song['liveness']
                        t_feature_matrix[ix][6] = song['loudness']
                        t_feature_matrix[ix][7] = song['mode']
                        t_feature_matrix[ix][8] = song['tempo']
                        t_feature_matrix[ix][9] = song['time_signature']
                        t_feature_matrix[ix][10] = song['valence']
                        t_feature_matrix[ix][11] = song['duration_ms']
                            
                    t_feature_median.append(np.percentile(t_feature_matrix,50,axis=0))
                    
                    # Part 1
                    
                    p1_ids = []
                    p1_popularities = []
                    p1_num_markets = []
                    
                    for ix, item in enumerate(p1):
                        p1_ids.append(item['track']['id'])
                        p1_popularities.append(item['track']['popularity'])
                        p1_num_markets.append(len(item['track']['available_markets']))

                    #p1_release_dates = []
                    #for ids in p1_ids:
                    #    track = sp.track(str(ids))
                    #    album = sp.album(track['album']['uri'])
                    #    p1_release_dates.append(int(album['release_date'][0:4]))
                    
                    #p1_release_date.append(np.mean(p1_release_dates))
                        
                    
                    p1_popularity.append(np.mean(p1_popularities))
                    p1_mean_num_markets.append(np.mean(p1_num_markets))
                    p1_features = sp.audio_features(tracks = p1_ids)
                    
                    p1_len = len(p1)
                    p1_feature_matrix = np.empty((p1_len,12))
                    for ix, song in enumerate(p1_features):
                    
                        p1_feature_matrix[ix][0] = song['acousticness']
                        p1_feature_matrix[ix][1] = song['danceability']
                        p1_feature_matrix[ix][2] = song['energy']
                        p1_feature_matrix[ix][3] = song['instrumentalness']
                        p1_feature_matrix[ix][4] = song['key']
                        p1_feature_matrix[ix][5] = song['liveness']
                        p1_feature_matrix[ix][6] = song['loudness']
                        p1_feature_matrix[ix][7] = song['mode']
                        p1_feature_matrix[ix][8] = song['tempo']
                        p1_feature_matrix[ix][9] = song['time_signature']
                        p1_feature_matrix[ix][10] = song['valence']
                        p1_feature_matrix[ix][11] = song['duration_ms']
                        
                    p1_feature_median.append(np.percentile(p1_feature_matrix,50,axis=0))
                    
                    # Part 2
                    
                    p2_ids = []
                    p2_popularities = []
                    p2_num_markets = []
                    
                    for ix, item in enumerate(p2):
                        p2_ids.append(item['track']['id'])
                        p2_popularities.append(item['track']['popularity'])
                        p2_num_markets.append(len(item['track']['available_markets']))

                    #p2_release_dates = []
                    #for ids in p2_ids:
                    #    track = sp.track(str(ids))
                    #    album = sp.album(track['album']['uri'])
                    #    p2_release_dates.append(int(album['release_date'][0:4]))
                    
                    #p2_release_date.append(np.mean(p2_release_dates))
                    
                    
                    p2_popularity.append(np.mean(p2_popularities))
                    p2_mean_num_markets.append(np.mean(p2_num_markets))
                    p2_features = sp.audio_features(tracks = p2_ids)
                    
                    p2_len = len(p2)
                    p2_feature_matrix = np.empty((p2_len,12))
                    for ix, song in enumerate(p2_features):
                    
                        p2_feature_matrix[ix][0] = song['acousticness']
                        p2_feature_matrix[ix][1] = song['danceability']
                        p2_feature_matrix[ix][2] = song['energy']
                        p2_feature_matrix[ix][3] = song['instrumentalness']
                        p2_feature_matrix[ix][4] = song['key']
                        p2_feature_matrix[ix][5] = song['liveness']
                        p2_feature_matrix[ix][6] = song['loudness']
                        p2_feature_matrix[ix][7] = song['mode']
                        p2_feature_matrix[ix][8] = song['tempo']
                        p2_feature_matrix[ix][9] = song['time_signature']
                        p2_feature_matrix[ix][10] = song['valence']
                        p2_feature_matrix[ix][11] = song['duration_ms']
                        
                    p2_feature_median.append(np.percentile(p2_feature_matrix,50,axis=0))
                    
                    # Part 3
                    
                    p3_ids = []
                    p3_popularities = []
                    p3_num_markets = []
                    
                    for ix, item in enumerate(p3):
                        p3_ids.append(item['track']['id'])
                        p3_popularities.append(item['track']['popularity'])
                        p3_num_markets.append(len(item['track']['available_markets']))

                    #p3_release_dates = []
                    #for ids in p3_ids:
                    #    track = sp.track(str(ids))
                    #    album = sp.album(track['album']['uri'])
                    #    p3_release_dates.append(int(album['release_date'][0:4]))
                    
                    #p3_release_date.append(np.mean(p3_release_dates))
                        
                    p3_popularity.append(np.mean(p3_popularities))
                    p3_mean_num_markets.append(np.mean(p3_num_markets))
                    p3_features = sp.audio_features(tracks = p3_ids)
                    
                    p3_len = len(p3)
                    p3_feature_matrix = np.empty((p3_len,12))
                    for ix, song in enumerate(p3_features):
                    
                        p3_feature_matrix[ix][0] = song['acousticness']
                        p3_feature_matrix[ix][1] = song['danceability']
                        p3_feature_matrix[ix][2] = song['energy']
                        p3_feature_matrix[ix][3] = song['instrumentalness']
                        p3_feature_matrix[ix][4] = song['key']
                        p3_feature_matrix[ix][5] = song['liveness']
                        p3_feature_matrix[ix][6] = song['loudness']
                        p3_feature_matrix[ix][7] = song['mode']
                        p3_feature_matrix[ix][8] = song['tempo']
                        p3_feature_matrix[ix][9] = song['time_signature']
                        p3_feature_matrix[ix][10] = song['valence']
                        p3_feature_matrix[ix][11] = song['duration_ms']
                        
                    p3_feature_median.append(np.percentile(p3_feature_matrix,50,axis=0))
                    
                    # Part 4
                    
                    p4_ids = []
                    p4_popularities = []
                    p4_num_markets = []
                    
                    for ix, item in enumerate(p4):
                        p4_ids.append(item['track']['id'])
                        p4_popularities.append(item['track']['popularity'])
                        p4_num_markets.append(len(item['track']['available_markets']))

                    #p4_release_dates = []
                    #for ids in p4_ids:
                    #    track = sp.track(str(ids))
                    #    album = sp.album(track['album']['uri'])
                    #    p4_release_dates.append(int(album['release_date'][0:4]))
                    
                    #p4_release_date.append(np.mean(p4_release_dates))
                    
                    p4_popularity.append(np.mean(p4_popularities))
                    p4_mean_num_markets.append(np.mean(p4_num_markets))
                    p4_features = sp.audio_features(tracks = p4_ids)
                    
                    p4_len = len(p4)
                    p4_feature_matrix = np.empty((p4_len,12))
                    for ix, song in enumerate(p4_features):
                    
                        p4_feature_matrix[ix][0] = song['acousticness']
                        p4_feature_matrix[ix][1] = song['danceability']
                        p4_feature_matrix[ix][2] = song['energy']
                        p4_feature_matrix[ix][3] = song['instrumentalness']
                        p4_feature_matrix[ix][4] = song['key']
                        p4_feature_matrix[ix][5] = song['liveness']
                        p4_feature_matrix[ix][6] = song['loudness']
                        p4_feature_matrix[ix][7] = song['mode']
                        p4_feature_matrix[ix][8] = song['tempo']
                        p4_feature_matrix[ix][9] = song['time_signature']
                        p4_feature_matrix[ix][10] = song['valence']
                        p4_feature_matrix[ix][11] = song['duration_ms']
                        
                    p4_feature_median.append(np.percentile(p4_feature_matrix,50,axis=0))

                    # Summary
                    
                    playlist_ids.append(playlist['id'])
                    followers.append(metadata['followers']['total'])
                    names.append(playlist['name'])
                    featured.append(playlist['name'] in featured_names)
                    total_tracks.append(playlist['tracks']['total'])
                    
                    popularity.append(np.mean(t_popularities))
                    t_mean_num_markets = np.mean(t_num_markets)
                    mean_num_markets.append(t_mean_num_markets)
                    
                    print("%4d %s" % (i + 1 + playlists['offset'], playlist['name']))
                    
            except:
                print 'error'
        if playlists['next']==None:
            break
    t_feature_median = np.array(t_feature_median)
    p1_feature_median = np.array(p1_feature_median)
    p2_feature_median = np.array(p2_feature_median)
    p3_feature_median = np.array(p3_feature_median)
    p4_feature_median = np.array(p4_feature_median)
    
    #t_release_date = np.array(t_release_date)
    #t_release_date = 2017 - t_release_date
    
    #p1_release_date = np.array(p1_release_date)
    #p1_release_date = 2017 - p1_release_date
    
    #p2_release_date = np.array(p2_release_date)
    #p2_release_date = 2017 - p2_release_date
    
    #p3_release_date = np.array(p3_release_date)
    #p3_release_date = 2017 - p3_release_date
    
    #p4_release_date = np.array(p4_release_date)
    #p4_release_date = 2017 - p4_release_date
    
    #print p4_release_date

    playlist_data = pd.DataFrame({
                    'followers': followers,
                    'names' : names,
                    'playlist_id' : playlist_ids,
                    'total_tracks' : total_tracks,
                    'featured' : featured,
                    'acousticness': p1_feature_median[:,0],
                    'danceability': p1_feature_median[:,1],
                    'energy': p1_feature_median[:,2],
                    'instrumentalness': p1_feature_median[:,3],
                    'key': p1_feature_median[:,4],
                    'liveness': p1_feature_median[:,5],
                    'loudness': p1_feature_median[:,6],
                    'mode': p1_feature_median[:,7],
                    'tempo': p1_feature_median[:,8],
                    'time_signature': p1_feature_median[:,9],
                    'valence': p1_feature_median[:,10],
                    'duration': p1_feature_median[:,11],
                    'popularity':popularity,
                    'mean_num_markets': mean_num_markets,
                    #'avg_years' : t_release_date,

                    'p1_acousticness': p1_feature_median[:,0],
                    'p1_danceability': p1_feature_median[:,1],
                    'p1_energy': p1_feature_median[:,2],
                    'p1_instrumentalness': p1_feature_median[:,3],
                    'p1_key': p1_feature_median[:,4],
                    'p1_liveness': p1_feature_median[:,5],
                    'p1_loudness': p1_feature_median[:,6],
                    'p1_mode': p1_feature_median[:,7],
                    'p1_tempo': p1_feature_median[:,8],
                    'p1_time_signature': p1_feature_median[:,9],
                    'p1_valence': p1_feature_median[:,10],
                    'p1_duration': p1_feature_median[:,11],
                    'p1_popularity': p1_popularity,
                    'p1_mean_num_markets': p1_mean_num_markets,
                    #'p1_avg_years': p1_release_date,

                    'p2_acousticness': p2_feature_median[:,0],
                    'p2_danceability': p2_feature_median[:,1],
                    'p2_energy': p2_feature_median[:,2],
                    'p2_instrumentalness': p2_feature_median[:,3],
                    'p2_key': p2_feature_median[:,4],
                    'p2_liveness': p2_feature_median[:,5],
                    'p2_loudness': p2_feature_median[:,6],
                    'p2_mode': p2_feature_median[:,7],
                    'p2_tempo': p2_feature_median[:,8],
                    'p2_time_signature': p2_feature_median[:,9],
                    'p2_valence': p2_feature_median[:,10],
                    'p2_duration': p2_feature_median[:,11],
                    'p2_popularity': p2_popularity,
                    'p2_mean_num_markets': p2_mean_num_markets,
                    #'p2_avg_years': p2_release_date,

                    'p3_acousticness': p3_feature_median[:,0],
                    'p3_danceability': p3_feature_median[:,1],
                    'p3_energy': p3_feature_median[:,2],
                    'p3_instrumentalness': p3_feature_median[:,3],
                    'p3_key': p3_feature_median[:,4],
                    'p3_liveness': p3_feature_median[:,5],
                    'p3_loudness': p3_feature_median[:,6],
                    'p3_mode': p3_feature_median[:,7],
                    'p3_tempo': p3_feature_median[:,8],
                    'p3_time_signature': p3_feature_median[:,9],
                    'p3_valence': p3_feature_median[:,10],
                    'p3_duration': p3_feature_median[:,11],
                    'p3_popularity': p3_popularity,
                    'p3_mean_num_markets': p3_mean_num_markets,
                    #'p3_avg_years': p3_release_date,

                    'p4_acousticness': p4_feature_median[:,0],
                    'p4_danceability': p4_feature_median[:,1],
                    'p4_energy': p4_feature_median[:,2],
                    'p4_instrumentalness': p4_feature_median[:,3],
                    'p4_key': p4_feature_median[:,4],
                    'p4_liveness': p4_feature_median[:,5],
                    'p4_loudness': p4_feature_median[:,6],
                    'p4_mode': p4_feature_median[:,7],
                    'p4_tempo': p4_feature_median[:,8],
                    'p4_time_signature': p4_feature_median[:,9],
                    'p4_valence': p4_feature_median[:,10],
                    'p4_duration': p4_feature_median[:,11],
                    'p4_popularity': p4_popularity,
                    'p4_mean_num_markets': p4_mean_num_markets
                    #'p4_avg_years': p4_release_date
            })

    #q.put(playlist_data)
    #final_playlists.append(playlist_data)

    #name = multiprocessing.current_process().name
    #print(name, '...Exiting')
    #print 'bye'
    return playlist_data

In [106]:
for offset in range(1701,1900):
    try:
        a = getPlaylistData(offset, 1)
        final_playlists.append(a)
        print 'appended'
    except:
        print 'error!!!'

Starting...
1702 Stockholm Music & Arts 2012
elapsed time: 1234.488964 ms 0.147967 projected finish
appended
Starting...
1703 Popaganda 2012
elapsed time: 1144.486904 ms 0.137179 projected finish
appended
Starting...
1704 Metaltown 2012
elapsed time: 1103.365898 ms 0.132251 projected finish
appended
Starting...
1705 Urkult 2012
elapsed time: 2084.856987 ms 0.249893 projected finish
appended
Starting...
1706 Sweden Rock Festival 2012
elapsed time: 1317.593098 ms 0.157928 projected finish
appended
Starting...
1707 Hultsfred 2012
elapsed time: 1465.134144 ms 0.175613 projected finish
appended
Starting...
1708 Putte i parken 2012
elapsed time: 2595.316172 ms 0.311077 projected finish
appended
Starting...
1709 Getaway Rock 2012
elapsed time: 1159.708023 ms 0.139004 projected finish
appended
Starting...
elapsed time: 248.291969 ms 0.029761 projected finish
error
error!!!
Starting...
1711 Sveriges Nationaldag
elapsed time: 1204.656124 ms 0.144391 projected finish
appended
Starting...
1712 Dan

//anaconda/lib/python2.7/site-packages/numpy/core/_methods.py:59: RuntimeWarning: Mean of empty slice.
  warnings.warn("Mean of empty slice.", RuntimeWarning)


elapsed time: 522.923946 ms 0.062678 projected finish
error
error!!!
Starting...
1732 Intervju med Mando Diao 
elapsed time: 1195.743799 ms 0.143323 projected finish
appended
Starting...
1733 Intervju med Agnes
elapsed time: 1108.587027 ms 0.132876 projected finish
appended
Starting...
1734 Intervju med Linnea Henriksson
elapsed time: 1075.181007 ms 0.128872 projected finish
appended
Starting...
1735 Intervju med Anders Fridén
elapsed time: 1093.248844 ms 0.131038 projected finish
appended
Starting...
1736 Intervju med Kristian Anttila
elapsed time: 1215.427876 ms 0.145683 projected finish
appended
Starting...
1737 Intervju med Veronica Maggio
elapsed time: 1071.435213 ms 0.128423 projected finish
appended
Starting...
1738 Hemma Hos Janne&Kjell
elapsed time: 1112.154961 ms 0.133304 projected finish
appended
Starting...
1739 Skratt till kaffet
elapsed time: 1155.802965 ms 0.138536 projected finish
appended
Starting...
elapsed time: 199.707031 ms 0.023937 projected finish
error
error!!!


In [107]:
final_playlists

[   acousticness  danceability  duration  energy featured  followers  \
 0        0.1325        0.6995  217744.5   0.592    False   14284402   
 
    instrumentalness  key  liveness  loudness   ...     p4_popularity  \
 0          0.000005  3.0     0.113    -5.026   ...         75.230769   
 
    p4_tempo p4_time_signature  p4_valence             playlist_id  popularity  \
 0   110.991               4.0       0.418  5FJXhjdILmRA2z5bvz4nzf       79.12   
 
       tempo  time_signature  total_tracks  valence  
 0  102.2015             4.0            50    0.491  
 
 [1 rows x 75 columns],
    acousticness  danceability  duration  energy featured  followers  \
 0        0.0318        0.7675  220976.5  0.5255    False    6061263   
 
    instrumentalness  key  liveness  loudness   ...     p4_popularity  \
 0          0.000032  3.0     0.107   -7.6295   ...         68.153846   
 
    p4_tempo p4_time_signature  p4_valence             playlist_id  popularity  \
 0   121.547               4.0

In [54]:
from multiprocessing import Process, Queue
import multiprocessing

if __name__ == '__main__':
    q = Queue()
    for offset in [0,400,800,1200,1600]:
    #for offset in [0]:

        Process(target=getPlaylistData, args=(offset,5,q)).start()
    #p.start()
    #print(q.get())    # prints "[42, None, 'hello']"
    #p.join()

Starting...
Starting...
Starting...
Starting...
Starting...
retrying ...7secs
retrying ...7secs
retrying ...7secs
retrying ...7secs
retrying ...7secs
retrying ...0secs
retrying ...0secs
retrying ...0secs
retrying ...0secs
retrying ...0secs
retrying ...0secs
retrying ...0secs
retrying ...0secs
retrying ...0secs
retrying ...0secs
retrying ...0secs
retrying ...0secs
retrying ...0secs
retrying ...0secs
retrying ...0secs
retrying ...0secs
retrying ...0secs
retrying ...0secs
retrying ...0secs
retrying ...0secs
retrying ...0secs
retrying ...0secs
retrying ...0secs


//anaconda/lib/python2.7/site-packages/numpy/lib/function_base.py:3834: RuntimeWarning: Invalid value encountered in percentile
  RuntimeWarning)


retrying ...8secs
retrying ...8secs
retrying ...8secs
retrying ...8secs
retrying ...8secs
retrying ...0secs
retrying ...0secs
retrying ...0secs
retrying ...0secs
retrying ...0secs
retrying ...0secs
retrying ...0secs
retrying ...0secs
retrying ...0secs
retrying ...0secs
retrying ...0secs
retrying ...0secs
retrying ...0secs
retrying ...0secs
retrying ...0secs
retrying ...0secs
retrying ...0secs
retrying ...0secs
retrying ...0secs
retrying ...0secs
retrying ...0secs
retrying ...0secs
1601 Meditate to Sounds of Nature
elapsed time: 18015.465975 ms 2.159354 projected finish
 801 Trick or Treat
   1 Today's Top Hits
elapsed time: 18043.498993 ms 2.162714 projected finish
elapsed time: 18087.514877 ms 2.167990 projected finish
 401 Summertime Vibes
elapsed time: 18100.445032 ms 2.169539 projected finish
1201 Soundscapes For Gaming
elapsed time: 18178.709030 ms 2.178920 projected finish
retrying ...8secs
retrying ...8secs
retrying ...8secs
retrying ...8secs
retrying ...8secs
retrying ...0secs


In [38]:
all_dfs = []
for _ in range(5):
    all_dfs.append(q.get())

KeyboardInterrupt: 

In [108]:
playlist_data = final_playlists[0]
for each in final_playlists[1:]:
    playlist_data = playlist_data.append(each,ignore_index=True)

In [111]:
playlist_data.head()

,acousticness,danceability,duration,energy,featured,followers,instrumentalness,key,liveness,loudness,...,p4_popularity,p4_tempo,p4_time_signature,p4_valence,playlist_id,popularity,tempo,time_signature,total_tracks,valence
0,0.1325,0.6995,217744.5,0.5920,False,14284402,0.000005,3.0,0.1130,-5.0260,...,75.230769,110.9910,4.0,0.4180,5FJXhjdILmRA2z5bvz4nzf,79.120000,102.2015,4.0,50,0.4910
1,0.0318,0.7675,220976.5,0.5255,False,6061263,0.000032,3.0,0.1070,-7.6295,...,68.153846,121.5470,4.0,0.3520,37i9dQZF1DX0XUsuxWHRQd,72.420000,142.0510,4.0,59,0.3110
2,0.1334,0.7275,226361.5,0.6060,True,2954185,0.000000,4.5,0.1035,-5.6395,...,55.000000,92.9790,4.0,0.3930,06CemleTteSalaVGVMbgFy,65.660000,127.4455,4.0,50,0.4855
3,0.0254,0.5820,214781.0,0.8185,False,2586720,0.000008,8.0,0.1665,-5.1860,...,58.307692,100.0430,4.0,0.3790,0lbtgFu3JNKX77J5YOpW7n,62.460000,123.4825,4.0,50,0.6320
4,0.8790,0.5530,216173.0,0.3170,False,2462696,0.003090,3.0,0.1100,-11.5270,...,51.150000,117.9865,4.0,0.3495,16BpjqQV1Ey0HeDueNDSYz,54.320513,124.8580,4.0,78,0.3130


In [114]:
playlist_data.to_csv("playlist_data_with_sequencing_4.8.17.csv",encoding='utf-8')

,acousticness,danceability,duration,energy,featured,followers,instrumentalness,key,liveness,loudness,...,p4_popularity,p4_tempo,p4_time_signature,p4_valence,playlist_id,popularity,tempo,time_signature,total_tracks,valence
0,0.013900,0.5260,252720.0,0.7070,False,70595,0.001610,4.0,0.1300,-8.6610,...,39.000000,122.8785,4.0,0.3225,37i9dQZF1DX8kI525v95Yu,44.461538,120.361,4.0,78,0.4820
1,0.394000,0.5645,262380.0,0.4320,False,3249,0.000487,7.0,0.1270,-10.7220,...,31.076923,127.4770,4.0,0.4650,37i9dQZF1DWUHyTgDVMaG1,38.720000,102.989,4.0,50,0.5335
2,0.000313,0.4635,255559.5,0.9125,False,1305,0.000122,3.5,0.1114,-3.9375,...,44.111111,114.5150,4.0,0.3700,37i9dQZF1DWYrXu9rm1zUZ,47.500000,109.051,4.0,34,0.4340


In [15]:
len(a.iloc[0])

80

In [16]:
a

,acousticness,avg_years,danceability,duration,energy,featured,followers,instrumentalness,key,liveness,...,p4_popularity,p4_tempo,p4_time_signature,p4_valence,playlist_id,popularity,tempo,time_signature,total_tracks,valence
0,0.013900,23.743590,0.5260,252720.0,0.7070,False,70593,0.001610,4.0,0.1300,...,39.000000,122.8785,4.0,0.3225,37i9dQZF1DX8kI525v95Yu,44.461538,120.361,4.0,78,0.4820
1,0.394000,34.160000,0.5645,262380.0,0.4320,False,3249,0.000487,7.0,0.1270,...,31.076923,127.4770,4.0,0.4650,37i9dQZF1DWUHyTgDVMaG1,38.720000,102.989,4.0,50,0.5335
2,0.000313,5.294118,0.4635,255559.5,0.9125,False,1302,0.000122,3.5,0.1114,...,44.111111,114.5150,4.0,0.3700,37i9dQZF1DWYrXu9rm1zUZ,47.500000,109.051,4.0,34,0.4340
